# Rechercher les meilleurs hyper-paramètres

In [1]:
import hashlib
import os
import shutil
import time
from dataclasses import dataclass
from typing import Callable
from matplotlib import pyplot as plt
pp=print
import numpy as np
import jax.numpy as jnp
import optax
import pickle
import jax
from jax.example_libraries import stax
from datetime import datetime

## Data

### Un problème de regression

On veut faire coller un modèle à une fonction toute simple.

In [2]:
n_data=20_000
X = jnp.linspace(0, 1, n_data)[:, None]
Y = jnp.sin(10 * X)
nb_data_train = int(n_data * 0.8)
idx=np.random.permutation(n_data)
X_=X[idx]
Y_=Y[idx]
X_train = X_[:nb_data_train]
Y_train = Y_[:nb_data_train]
X_val = X_[nb_data_train:]
Y_val = Y_[nb_data_train:]

In [3]:
fig,ax=plt.subplots()
ax.plot(X,Y);

### Distributeur de batch

On va distribuer les données avec une double boucle. Une boucle sur les époques. A chaque époque les données sont réparties dans les batchs.

In [4]:
def batchs_for_one_epoch(X_all,Y_all,batch_size):
    nb_batches=len(X_all)//batch_size
    shuffle_index=np.random.permutation(len(X_all))
    X_all_shuffle=X_all[shuffle_index]
    Y_all_shuffle=Y_all[shuffle_index]
    for i in range(nb_batches):
        yield X_all_shuffle[i*batch_size:(i+1)*batch_size],Y_all_shuffle[i*batch_size:(i+1)*batch_size]

## Modèle et fonctions d'entrainement

### Un MLP en stax




Un réseau neuronal est une fonction paramétrique $f(\theta,x)$. En jax, on l'implémente très explicitement: en se donnant une fonction `model_init()` qui renverra un jeu initial de paramètre `model_param` =$\theta$  et la fonction `model_call(model_param,x)` =$f(\theta,x)$.

Ici on va utiliser la librarie 'stax' qui est ultra-simple. Elle est considérer comme une librairie  'exemple'.

In [5]:
stax_activation_layers={
    "tanh":stax.Tanh,"relu":stax.Relu, "exp": stax.Exp,"log_softmax": stax.LogSoftmax,
    "softmax": stax.Softmax, "softplus": stax.Softplus, "sigmoid": stax.Sigmoid, "elu": stax.Elu,
   "leaky_relu": stax.LeakyRelu, "selu": stax.Selu, "gelu": stax.Gelu
}

#On va utiliser un Multi-layer-perceptron.
def MLP_stax(dim_in,dim_hidden,dim_out,n_layer,*,activation_name="relu"):
    layers = []
    activation_layer=stax_activation_layers[activation_name]
    for i in range(n_layer - 1):
        layers.append(stax.Dense(dim_hidden))
        layers.append(activation_layer)
    layers.append(stax.Dense(dim_out))
    model_init_stax, model_call= stax.serial(*layers)

    def model_init(rand_key=None):
        if rand_key is None:
            rand_key=jax.random.key(int(datetime.now().timestamp()*1_000_000))
        _, params = model_init_stax(rand_key, (-1, dim_in))
        return params

    return model_init,model_call

In [6]:
model_init,model_call=MLP_stax(dim_in=1,dim_hidden=64,dim_out=1,n_layer=3)
model_param=model_init()
Y_pred=model_call(model_param,X)

fig,ax=plt.subplots()
ax.plot(X,Y_pred);

Sans entrainement, la courbe est aléatoire.

Ensuite, il faut trouver le bon paramètre $\theta$ pour que le réseau neuronal colle aux données. Cette recherche se fait à l'aide d'une descente de gradient.

Mais pour réussir au mieux, il faut aussi bien calibrer l'architecture du réseau de neurone (`n_layer`, `hidden_dim`, `activation_name`) paramètres de l'optimisation (`learning_rate`, `batch_size`, etc.). On nommera tous ces paramètres "hyper-paramètres" pour les différencier des paramètres du modèles.




Nous cherchons à organiser notre code pour pouvoir tester différents modèles et différents modes d'apprentissages. On aimerait trouver le bon équilibre entre "tout automatiser" et  "tout faire à la main".

On aimerait aussi que les paramètres des modèles entrainés soient sauvegardés pour pouvoir prolonger un entrainement. Et on veut aussi se souvenir de tous les hyper-paramètres qui ont été testé.

### De l'Hyper-paramètre au modèle

In [ ]:
def make_model(hyper_param):
    model_init,model_call=MLP_stax(dim_in=1, dim_hidden=hyper_param["layer_size"], dim_out=1, n_layer=hyper_param["n_layer"], activation_name=hyper_param["activation_name"])
    return model_init,model_call


### fonction loss et update

In [ ]:
def jit_creator(model_call,optimizer):
    @jax.jit
    def loss_compute(params, X,Y):
        Y_pred=model_call(params,X)
        return jnp.mean((Y_pred-Y)**2)

    @jax.jit
    def update_model_param(optimizer_state, model_param, X,Y):
        grads = jax.grad(loss_compute)(model_param, X,Y)
        updates, optimizer_state = optimizer.update(grads, optimizer_state)
        #here the model_param is modified
        model_param = optax.apply_updates(model_param, updates)
        return optimizer_state, model_param

    return loss_compute,update_model_param

## L'Agent

J'utilise le mot 'Agent' pour un objet qui permet l'entrainement.

J'ai l'impression que maintenant la plupart des gens utilisent le mot 'Trainer'.


Appelons la technique que l'on va utiliser 'ful-folder'. Tout ce qui est produit durant l'entrainement est sauvé dans un folder.

### Fonctions de sauvegarde

In [7]:
def save_as_pickle(file_name,serializable):
    pickle.dump(serializable,open(file_name,"wb"))
def load_from_pickle(file_name):
    return pickle.load(open(file_name,"rb"))
def save_as_str(file_name,serializable):
    with open(file_name, "wt") as f:
        f.write(str(serializable))
def load_from_str(file_name):
    with open(file_name, "rt") as f:
        res = eval(f.read())
    return res

### L'Agent en personne

In [11]:
@dataclass
class AgentMiniResult:
    hyper_param:dict
    best_loss:float
    model_param:dict
    model_call:Callable


class AgentMini:
    @staticmethod
    def load(folder):
        assert os.path.exists(folder),f"folder:{folder} does not exist"
        model_param = load_from_pickle(f"{folder}/model_param")
        best_loss = load_from_str(f"{folder}/best_loss")
        model_param=load_from_pickle(f"{folder}/model_param")
        hyper_param=load_from_str(f"{folder}/hyper_param")
        _, model_call = make_model(hyper_param)
        return AgentMiniResult(hyper_param,best_loss,model_param,model_call)

    @staticmethod
    def train(folder,hyper_param,n_epoch,verbose):

        model_init, model_call = make_model(hyper_param)
        optimizer = optax.adam(hyper_param["learning_rate"])
        batch_size = hyper_param["batch_size"]

        if os.path.exists(folder):
            if verbose:
                print(f"Existing folder:{folder}, we load model_param and best_loss from if")
            model_param = load_from_pickle(f"{folder}/model_param")
            best_loss =load_from_str(f"{folder}/best_loss")

        else:#c'est la première fois qu'on teste cet hyper_paramètre.
            os.makedirs(folder, exist_ok=True)
            if verbose:
                print(f"New folder:{folder}, model_param are randomly initialized")
            model_param=model_init()
            best_loss=1e10
            save_as_pickle(f"{folder}/model_param", model_param)
            save_as_str(f"{folder}/best_loss", best_loss)

        save_as_str(f"{folder}/hyper_param", hyper_param)
        optimizer_state=optimizer.init(model_param)
        loss_compute, update_model_param=jit_creator(model_call,optimizer)

        for _ in range(n_epoch):
            for x,y in batchs_for_one_epoch(X_train,Y_train,batch_size):
                optimizer_state, model_param = update_model_param(optimizer_state, model_param, x,y)

            val_loss=float(loss_compute(model_param, X_val, Y_val))
            if val_loss <= best_loss:
                best_loss=val_loss
                if verbose:
                    print(f"⬊{val_loss:.3g}", end="")
                save_as_pickle(f"{folder}/model_param",model_param)
                save_as_str(f"{folder}/best_loss",best_loss)
            else:
                if verbose:
                    print(".",end="")
        if verbose:
            print("| end of the optimization loop.")
        return best_loss

## Les entrainements

### Entrainement pour 1 hyper-paramètre



In [16]:
def one_training(folder):
    # noinspection PyDictCreation
    hyper_param = {"n_layer": 2, "layer_size": 32,"batch_size":512,"learning_rate":1e-3,"activation_name":"relu"}

    #Pour que ce ne soit pas un ré-entrainement.
    shutil.rmtree(folder,ignore_errors=True)
    AgentMini.train(folder,hyper_param,20,verbose=True)

In [18]:
folder="data/mon_premier_test"
one_training(folder)

In [17]:
def eval_training(folder):
    result=AgentMini.load(folder)
    fig, axs = plt.subplots(1, 1, sharex="all")
    Y_pred=result.model_call(result.model_param,X)
    axs.plot(X,Y_pred)
    axs.plot(X,Y)
    fig.tight_layout()
    plt.show()

In [19]:
eval_training(folder)

⇑ pas terrible

### Une rechercher en grille

In [24]:
def manual_loop(mother_folder):
    # noinspection PyDictCreation
    hyper_params = [
        {"n_layer": 2, "layer_size": 32},
        {"n_layer": 3, "layer_size": 32},
        {"n_layer": 4, "layer_size": 32}
    ]
    for hyper_param in hyper_params:
        hyper_param["batch_size"] = 512
        # noinspection PyTypeChecker
        hyper_param["learning_rate"] = 1e-3
        # noinspection PyTypeChecker
        hyper_param["activation_name"] = "relu"

    for i,hyper_param in enumerate(hyper_params):
        folder=f"{mother_folder}/test_{i}"
        shutil.rmtree(folder,ignore_errors=True)
        AgentMini.train(folder,hyper_param,20,verbose=True)

In [25]:
manual_loop("data/manual_loop")

In [26]:
def hyper_param_to_str(hyper_param):
    return f"bs:{hyper_param['batch_size']},n_lay:{hyper_param['n_layer']},lay_size:{hyper_param['layer_size']},lr:{hyper_param['learning_rate']:.3g}"


In [27]:
def eval_trainings(mother_folder):
    folders=list(os.listdir(mother_folder))
    loss_results=[]
    for i,_folder in enumerate(folders):
        folder=f"{mother_folder}/{_folder}"
        result=AgentMini.load(folder)
        loss_results.append((result.best_loss,result))
    loss_results=sorted(loss_results,key=lambda a:a[0])
    results=[a[1] for a in loss_results]
    ni = 3
    some_results=[results[0],results[len(results)//2],results[-1]]
    fig, axs = plt.subplots(ni, 1, sharex="all", figsize=(6, ni * 4))
    titles=["best","intermediate","worst"]

    for i,result in enumerate(some_results):
        Y_pred=result.model_call(result.model_param,X)
        axs[i].plot(X,Y_pred)
        axs[i].plot(X,Y)
        axs[i].set_title(titles[i]+':'+hyper_param_to_str(result.hyper_param))
    fig.tight_layout()
    plt.show()

In [28]:
eval_trainings("data/manual_loop")

### Utilisation d'une libraire d'optimisation

Hyper-op est une librairie d'optimisation bayesienne. Ce genre d'optimisation est utile lorsque vous essayez de trouver le minimum (ou le maximum) d'une fonction dont l'évaluation prend du temps et dont vous ne connaissez pas la forme exacte (la "boîte noire").

Ici on cherche le minimum de la fonction qui à un hyper-paramètre associe la 'best-loss' obtenue pendant l'entrainement.

In [30]:
def hyperop_loop(mother_folder):
    n_epoch=20
    def objective(hyper_param):
        #un nome de fichier par rapport au temps
        #attention,il faut que les trainings durent plus d'une seconde
        n_second=str(time.time()).split(".")[0][5:]
        folder=f"{mother_folder}/test_{n_second}"
        return AgentMini.train(folder,hyper_param,n_epoch,False)

    from hyperopt import hp,fmin, tpe, space_eval
    space = {
        'batch_size': hp.choice('batch_size', [128,256,512]),
        'n_layer': hp.choice('n_layer', [2,3,4]),
        'layer_size': hp.choice('layer_size', [64,128,256]),
        'activation_name':'relu',
        'learning_rate':hp.loguniform('learning_rate',np.log(5e-4),np.log(1e-2))
    }
    best = fmin(objective, space, algo=tpe.suggest, max_evals=10)
    print("best hyper_parameter found:")
    print(space_eval(space, best))

In [31]:
hyperop_loop("data/hyperop_loop")

In [32]:
eval_trainings("data/hyperop_loop")

### Optimisation Bayésienne (explication mathématique)


Par Gemini (à destinations des plus téméraires, ne sera pas évalué).


L'optimisation bayésienne est une technique d'optimisation séquentielle pour les fonctions boîte noire coûteuses à évaluer. On cherche à minimiser une fonction inconnue $f(x)$, où $x$ est un vecteur d'hyperparamètres et $f(x)$ est la valeur de la fonction objectif (par exemple, la perte de validation) pour ces hyperparamètres. L'évaluation de $f(x)$ est coûteuse.

1.  **Modèle de substitution (Surrogate Model):** On utilise généralement un processus Gaussien (GP) pour modéliser $f(x)$. Un GP est caractérisé par une moyenne $\mu(x)$ et une covariance (ou noyau) $k(x, x')$. Après avoir évalué la fonction à $n$ points $D_n = \{(x_i, y_i)\}_{i=1}^n$, où $y_i = f(x_i)$, le GP nous donne une distribution prédictive sur la valeur de $f$ à un nouveau point $x^*$. Cette distribution prédictive est une distribution normale avec une moyenne $\mu_n(x^*)$ et une variance $\sigma_n^2(x^*)$, qui dépendent des données observées $D_n$ et de la fonction de covariance choisie.
$$
P(f(x^*) | D_n) = \mathcal{N}(\mu_n(x^*), \sigma_n^2(x^*))
$$
La moyenne $\mu_n(x^*)$ représente la meilleure estimation de $f(x^*)$ basée sur les données observées, et la variance $\sigma_n^2(x^*)$ mesure l'incertitude de cette estimation.




2.  **Fonction d'acquisition (Acquisition Function):** Une fonction d'acquisition $a(x)$ est définie en utilisant le modèle de substitution pour quantifier l'intérêt d'évaluer la fonction boîte noire au point $x$. L'objectif est de maximiser $a(x)$. Une fonction d'acquisition courante est l'**amélioration attendue (Expected Improvement - EI)**. Si $y_{min}$ est la meilleure valeur observée jusqu'à présent ($y_{min} = \min \{y_i\}_{i=1}^n$), l'amélioration au point $x$ est définie comme $I(x) = \max(y_{min} - f(x), 0)$. L'EI est l'espérance de cette amélioration par rapport à la distribution prédictive du GP sur $f(x)$:
$$
EI(x) = E[\max(y_{min} - f(x), 0) | D_n]
$$
L'EI peut être calculée analytiquement pour un GP. Si $f(x) \sim \mathcal{N}(\mu_n(x), \sigma_n^2(x))$, alors :
$$
EI(x) = \sigma_n(x) (\phi(Z) + Z\Phi(Z))
$$
où $Z = \frac{y_{min} - \mu_n(x)}{\sigma_n(x)}$, $\phi$ est la fonction de densité de probabilité de la loi normale standard, et $\Phi$ est sa fonction de répartition cumulative.
La fonction EI favorise les points où la moyenne prédictive est faible (potentiellement de nouvelles meilleures valeurs) et les points où l'incertitude est élevée (exploration de régions inconnues).


3.  **Processus d'optimisation:**
    *   Initialiser avec quelques points d'évaluation.
    *   À chaque itération :
        *   Mettre à jour le GP basé sur toutes les données évaluées.
        *   Trouver le point $x_{next}$ qui maximise la fonction d'acquisition $a(x)$. Ceci est un problème d'optimisation plus simple que l'optimisation de $f(x)$ car nous avons une expression analytique pour $a(x)$ et ses dérivées.
        *   Évaluer la fonction boîte noire au point $x_{next}$ pour obtenir $y_{next} = f(x_{next})$.
        *   Ajouter $(x_{next}, y_{next})$ aux données évaluées $D_n$.
    *   Répéter jusqu'à la convergence ou un nombre maximal d'itérations.

En résumé, l'optimisation bayésienne utilise un modèle probabiliste pour approximer la fonction coûteuse à évaluer et une fonction d'acquisition pour choisir intelligemment les points à évaluer ensuite, en équilibrant exploration et exploitation. Cela permet de trouver de bons optimums avec un nombre d'évaluations de la fonction boîte noire souvent beaucoup plus faible qu'avec d'autres méthodes.

##Le défi prog


* Une amélioration possible dans l'agent est de stocké aussi l'optimisateur-state. Ainsi si l'on redémarre l'entrainement, l'optimiseur sera déjà échauffé.


* Notre algorithme a un défaut: si la librairie hyperop tire 2 fois un même hyper-paramètre, alors il sera ré-entrainé. Il va alors avoir un super score artificiellement. Corrigez ce problème.


* Cette agent 'Mini' est fait pour être copié-coller dans vos différents projets et améliré. Par exemple, il pourrait aussi stocké le temps pris par l'entrainement. Et on pourrait alors ajouter un score 'loss*temps-d'entrainement' qui serait plus honnète pour les petits modèles. Ou alors on peut faire les entrainements au chrono (ex: 1 minute chacun).


Choisissez l'item ci-dessus qui vous inspire le plus, et codez-le.



